In [313]:
import pandas as pd
import subprocess
import math
import glob
from Bio import SeqIO
from tRNA_position import *
pd.set_option('display.max_colwidth',10000)
pd.set_option('display.width', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows',1000)
isotypes = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Glu', 'Gly', 'His', 'Ile', 'iMet', 'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val']

## Introduction

A global view of identity elements versus biological features would be a powerful tool for predicting tRNA function using primary sequence. To do this, I'll need to align eukaryotic tRNAs to our established models and annotate positions based on universal numbering. Then, I'll extract sequence information such as position, clade, or variable arm length.

## Process tRNAs
### Species information

In [3]:
species_table = pd.read_table('/projects/lowelab/users/blin/tRNAscan/models/taxonomy/genome_table_20151017.txt', header=None, names=['shortname', 'name', 'longname', 'domain', 'clade', 'subclade'])
# set NaN to empty strings
species_table.ix[species_table.name.apply(lambda x: type(x) is float), 'name'] = ''
# special cases
species_table.ix[species_table.shortname == "Bdist", 'name'] = "braDis1"
species_table.ix[species_table.shortname == "Gmax2", 'name'] = "glyMax2"
species_table.ix[species_table.shortname == "Mtrun", 'name'] = "mt3"
species_table.ix[species_table.shortname == "Phama1", 'name'] = "papHam1"
species_table.ix[species_table.shortname == "Ppate", 'name'] = "phyPat1.1"
species_table.ix[species_table.shortname == "Ptric", 'name'] = "popTri2"
species_table.ix[species_table.shortname == "Vvini", 'name'] = "vitVin1"
species_table.ix[species_table.shortname == "Zmays5", 'name'] = "zeaMay5"
species_table.head()

,shortname,name,longname,domain,clade,subclade
0,Pfalc,plasFalc1,Plasmodium falciparum (Oct 2007),eukaryota,Apicomplexa,NaN
1,Ppate,phyPat1.1,Physcomitrella patens (Version 1.1),eukaryota,Bryophyta,NaN
2,Spurp,,Strongylocentrotus purpuratus (Sea urchin) (Version 2.1),eukaryota,Echinodermata,NaN
3,Lmajo,,Leishmania major (Version 5.0),eukaryota,Euglenozoa,NaN
4,Ashb_goss_ATCC_10895,ashbGoss_ATCC10895,Ashbya gossypii ATCC 10895,eukaryota,Fungi,NaN


### Align tRNAs to existing CMs

We have a "quality set" of tRNAs, which include tRNAs from previous versions of genomes and newer versions of genomes. It is simpler and easier to create a new fasta file and alignment for each isotype, based on the `.iso` files from each run.

I also need to manually curate the genome table file to include the new species. And while I'm doing that, I might as well fix the broken entries.

In [ ]:
species = glob.glob("iso/*.iso")
species = [sp[4:-10] for sp in species]
isotype_files = {isotype:open('fasta/euk-' + isotype + '.fa', 'w') for isotype in isotypes} # so we can write more than one sequence at a time
for sp in species:
  iso_scores = pd.read_table('iso/' + sp + '-tRNAs.iso', header=0)
  iso_scores['best'] = iso_scores.ix[:,2:].idxmax(axis=1)
  for seq in SeqIO.parse('tRNAs/' + sp + '-tRNAs.fa', 'fasta'):
    isotype = iso_scores[iso_scores.tRNAscanID == seq.name].best.values[0]
    seq.id = sp + '_' + seq.description
    seq.description = ''
    SeqIO.write(seq, isotype_files[isotype], 'fasta')


for isotype in isotypes: isotype_files[isotype].close()



### Parse alignment positions

In [38]:
def position_base(positions, seq):
  for position_index, position in enumerate(positions):
    if position.paired:
      index1, index2 = position.position.split(':')
      index1, index2 = int(index1), int(index2)
      base_pair = "{}:{}".format(seq[index1 - 1], seq[index2 - 1])
      yield position.sprinzl, base_pair
    else:
      index = int(position.position)
      base = seq[index - 1]
      yield position.sprinzl, base
      
isotypes = ['Ala']
identities = pd.DataFrame()

for isotype in isotypes:
  alignment = 'alignments/euk-' + isotype + '.sto'
  # get positions
  alignment_fhandle = open(alignment)
  positions = [] # list containing each position in the tRNA
  # first, get secondary structure
  for line in alignment_fhandle:
    if line[0:12] == '#=GC SS_cons':
      ss = line.strip().split()[-1]
  # parse secondary structure into regions and positions
  positions = annotate_positions(ss)
  # get counts for each position by parsing Stockholm file
  alignment_fhandle = open(alignment)
  for line in alignment_fhandle:
    if line[0] in ["#", '\n', '/']: continue
    seqname, seq = line.strip().split()
    species = re.split('_|\\.trna', seqname, 1)[0]
    if any(species_table.shortname == species): row = species_table[species_table.shortname == species]
    elif any(species_table.name == species): row = species_table[species_table.name == species]
    elif any(species_table.name.apply(lambda x: bool(re.search(species, x)))): row = species_table[species_table.name.apply(lambda x: bool(re.search(species, x)))]
    else: continue # skip species that need too much babying
    domain = row.domain.values[0]
    clade = row.clade.values[0]
    species_long = row.longname.values[0]
    trna = {'domain': domain, 'clade': clade, 'species': species, 'species_long': species_long, 'seqname': seqname, 'isotype': isotype}
    trna = {**trna, **{sprinzl: base for sprinzl, base in position_base(positions, seq)}}
    identities = identities.append([trna], ignore_index=True)

identities.head()

,10:25,11:24,12:23,13:22,14,14i1,15,16,17,17i1,18,19,1:72,1i1,20,20i1,20i2,21,22i1,26,27:43,27i1,28:42,29:41,2:71,30:40,31:39,32,33,34,35,35i1,36,37,37i1,37i10,37i11,37i12,37i13,37i14,37i15,37i16,37i17,37i18,37i19,37i2,37i20,37i21,37i22,37i23,37i24,37i25,37i26,37i27,37i28,37i29,37i3,37i30,37i31,37i32,37i33,37i34,37i35,37i36,37i37,37i38,37i39,37i4,37i40,37i41,37i42,37i43,37i44,37i45,37i46,37i47,37i48,37i49,37i5,37i50,37i51,37i52,37i53,37i54,37i55,37i56,37i57,37i58,37i59,37i6,37i60,37i61,37i62,37i63,37i64,37i65,37i66,37i7,37i8,37i9,38,38i1,3:70,3i1,3i2,44,44i1,44i2,44i3,44i4,45,47,47i1,47i2,48,49:65,49i1,4:69,50:64,50i1,51:63,51i1,52:62,53:61,54,54i1,55,56,57,58,59,59i1,5:68,5i1,5i2,60,64i1,64i2,65i1,65i2,6:67,6i1,70i1,70i2,72i1,72i2,72i3,7:66,7i1,7i2,7i3,7i4,7i5,8,9,V1,V11:V21,V12:V22,V13:V23,V14:V24,V15:V25,V16:V26,V17:V27,V2,V3,V4,clade,domain,isotype,seqname,species,species_long
0,G:C,C:G,U:A,C:U,A,.,G,U,-,.,G,G,G:C,.,U,.,.,A,.,G,C:G,.,U:G,C:U,G:U,G:G,C:U,U,U,N,N,.,N,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,U,.,G:C,.,.,A,.,.,.,.,A,G,.,u,C,C:G,.,G:C,C:G,.,G:C,.,G:C,G:C,U,.,U,C,A,A,U,.,A:G,.,.,C,.,.,.,.,C:G,.,.,.,.,.,.,G:C,.,.,.,.,.,U,A,-,-:-,-:-,-:-,-:-,-:-,-:-,-:-,-,-,-,Insecta,eukaryota,Ala,Agamb_chr2R.trna81-AlaCGC,Agamb,Anopheles gambiae
1,G:C,C:G,U:A,C:U,A,.,G,U,-,.,G,G,G:C,.,U,.,.,A,.,G,U:A,.,U:G,C:U,G:U,G:G,C:U,U,U,N,N,.,N,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,U,.,G:C,.,.,A,.,.,.,.,G,G,.,c,C,C:G,.,G:C,C:G,.,G:C,.,G:C,G:C,U,.,U,C,G,A,U,.,A:G,.,.,C,.,.,.,.,U:G,.,.,.,.,.,.,G:C,.,.,.,.,.,U,A,-,-:-,-:-,-:-,-:-,-:-,-:-,-:-,-,-,-,Insecta,eukaryota,Ala,Agamb_chr2R.trna114-AlaTGC,Agamb,Anopheles gambiae
2,G:C,C:G,U:A,C:U,A,.,G,U,-,.,G,G,G:C,.,U,.,.,A,.,G,U:A,.,U:G,C:U,G:U,G:G,C:U,U,U,N,N,.,N,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,U,.,G:C,.,.,A,.,.,.,.,G,G,.,c,C,C:G,.,G:U,C:G,.,G:C,.,G:C,G:C,U,.,U,C,G,A,U,.,A:G,.,.,C,.,.,.,.,U:G,.,.,.,.,.,.,A:C,.,.,.,.,.,U,A,-,-:-,-:-,-:-,-:-,-:-,-:-,-:-,-,-,-,Insecta,eukaryota,Ala,Agamb_chr2R.trna16-AlaTGC,Agamb,Anopheles gambiae
3,G:C,C:G,U:A,C:U,A,.,G,U,-,.,G,G,G:C,.,U,.,.,A,.,G,C:G,.,U:G,C:U,G:U,G:G,C:U,U,U,N,N,.,N,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,U,.,G:C,.,.,A,.,.,.,.,A,G,.,u,C,C:G,.,G:C,C:G,.,G:C,.,G:C,G:C,U,.,U,C,A,A,U,.,A:G,.,.,C,.,.,.,.,U:G,.,.,.,.,.,.,G:C,.,.,.,.,.,U,A,-,-:-,-:-,-:-,-:-,-:-,-:-,-:-,-,-,-,Insecta,eukaryota,Ala,Agamb_chr2L.trna13-AlaCGC,Agamb,Anopheles gambiae
4,G:C,C:G,U:A,C:U,A,.,G,A,U,.,G,G,G:C,.,U,.,.,A,.,G,C:G,.,U:G,C:U,G:U,G:G,C:U,U,U,N,N,.,N,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,U,.,G:C,.,.,A,.,.,.,.,G,G,.,u,A,C:G,.,G:C,C:G,.,G:C,.,G:C,G:C,A,.,U,C,G,A,U,.,G:G,.,.,A,.,.,.,.,U:G,.,.,.,.,.,.,G:C,.,.,.,.,.,U,A,-,-:-,-:-,-:-,-:-,-:-,-:-,-:-,-,-,-,Insecta,eukaryota,Ala,Agamb_chr2L.trna126-AlaAGC,Agamb,Anopheles gambiae


In [165]:
# covariance model doesn't contain base 48. It's annotated as position 47i2 instead.

## Additional sequence information

In [ ]:
identities['anticodon'] = identities.seqname.apply(lambda x: x[-3:])
identities['isotype_ac'] = identities.seqname.apply(lambda x: x[-6:-3])

# GC content
paired_cols = identities.columns[list(map(lambda x: (':' in x), identities.columns))]
identities['GC'] = identities[paired_cols].apply(lambda x: sum((x == "G:C") | (x == "C:G"))/len(paired_cols), axis=1)

### Loop sizes

In [191]:
def bounds_to_cols(cols, start, end):
  selected_cols = []
  for col in cols:
    matches = re.findall('\d+', col)
    if len(matches) < 1: continue
    index = int(matches[0])
    if (index >= start and index <= end or col[0:3] == '{}i'.format(start - 1)) and col[0] != 'V':
      selected_cols.append(col)
  return selected_cols

dloop_cols = bounds_to_cols(identities.columns, 14, 21)
identities['D-loop'] = identities[dloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

acloop_cols = bounds_to_cols(identities.columns, 32, 38)
identities['AC-loop'] = identities[acloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

tpcloop_cols = bounds_to_cols(identities.columns, 54, 60)
identities['TPC-loop'] = identities[tpcloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

varm_cols = list(filter(lambda x: 'V' in x, identities.columns))
identities['V-arm'] = identities[varm_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

### Insertions/deletions

In [ ]:
# Insertions
insertion_cols = list(filter(lambda x: re.match('\d+i', x), identities.columns))
identities['insertions'] = identities[insertion_cols].apply(lambda x: sum(x != '.'), axis=1)

# Deletions at positions that are not the variable arm
base_cols = list(filter(lambda x: re.match('^((\d+)|(\d+:\d+))$', x), identities.columns))
identities['deletions'] = identities[base_cols].apply(lambda x: ''.join(x).count('-'), axis=1)


#### Get high-quality tRNA set

There was talk about using the high quality tRNAs to look at identity elements. Instead, I will just annotate specific tRNAs as high-quality or not. We can sort out the differences later.

In [281]:
# Import list of quality tRNAs
quality_trnas = []
quality_set = pd.read_table('quality-set.tsv', header=None, names=['organism', 'chrom', 'start', 'end'])
quality_set['organism'] = quality_set.organism.apply(lambda x: re.split('_|\\.trna', x, 1)[0])
for isotype in isotypes:
  alignment = 'alignments/euk-' + isotype + '.sto'
  # get positions
  for line in open(alignment):
    if line[0:4] != "#=GS": continue
    _, seqname, _, bounds, _, _, _, _, _, _ = line.strip().split()
    species = re.split('_|\\.trna', seqname, 1)[0]
    start, end = [int(bound) for bound in re.findall('\d+', bounds)]
    if not any(quality_set.organism == species):
      if any(species_table.shortname == species): species = species_table[species_table.shortname == species].name.values[0]
      elif any(species_table.name == species): species = species_table[species_table.name == species].shortname.values[0]
      elif any(species_table.name.apply(lambda x: bool(re.search(species, x)))): species = species_table[species_table.name.apply(lambda x: bool(re.search(species, x)))].name.values[0]
    if not any(quality_set.organism == species):
      print("skip", seqname)
      continue # skip species that don't exist or need too much babying
    
    num_matched_trnas = sum((quality_set.organism == species) & (quality_set.chrom.apply(lambda x: x in seqname)) & (abs(quality_set.start - start) < 10) & (abs(quality_set.end - end) < 10))
    if num_matched_trnas == 0:
      print("no match", seqname)
      continue
    elif num_matched_trnas > 1:
      print("multimatch", seqname)
      continue
    
    quality_trnas.append(seqname)

skip Agamb_chr2R.trna81-AlaCGC
skip Agamb_chr2R.trna114-AlaTGC
skip Agamb_chr2R.trna16-AlaTGC
skip Agamb_chr2L.trna13-AlaCGC
skip Agamb_chr2L.trna126-AlaAGC
skip Agamb_chr3R.trna35-AlaAGC
skip Agamb_chr3R.trna14-AlaAGC
skip Agamb_chr3R.trna51-AlaAGC
no match araTha1_chr2.trna89-GlyGCC
no match balAcu1_KI538153.trna1333-AlaAGC
no match bosTau8_chr6.trna8737-CysGCA
no match bosTau8_chr9.trna6204-TyrGTA
skip braDis1_chr1.trna18-AlaAGC
skip braDis1_chr2.trna98-AlaTGC
skip braDis1_chr4.trna57-AlaTGC
skip braDis1_chr3.trna37-AlaCGC
skip braDis1_chr1.trna119-AlaCGC
skip braDis1_chr1.trna120-AlaAGC
skip braDis1_chr4.trna90-AlaAGC
skip braDis1_scaffold_6.trna18-AspGTC
skip braDis1_chr3.trna98-GlyGCC
no match caeJap1_chrUn.trna480-AlaTGC
no match caeRem3_chrUn.trna507-AlaTGC
no match calMil1_KI636042.trna41-AlaTGC
no match calMil1_KI635860.trna304-AlaTGC
no match calMil1_KI635855.trna778-AlaTGC
no match calMil1_KI635862.trna287-AlaCGC
no match calMil1_KI635870.trna18-AlaCGC
no match calMil1_KI63

In [280]:
quality_trnas

[    organism       chrom   start     end
 257  ailMel1  GL193666.1  155406  155335,
     organism       chrom   start     end
 291  ailMel1  GL193897.1  200837  200908,
     organism       chrom   start     end
 301  ailMel1  GL193897.1  302892  302963,
     organism       chrom   start     end
 289  ailMel1  GL193897.1  191336  191407,
     organism       chrom   start     end
 311  ailMel1  GL193897.1  241046  240975,
     organism       chrom   start     end
 287  ailMel1  GL193897.1  175733  175804,
     organism       chrom   start     end
 309  ailMel1  GL193897.1  311269  311198,
     organism       chrom   start     end
 299  ailMel1  GL193897.1  272870  272941,
     organism       chrom   start     end
 312  ailMel1  GL193897.1  238784  238713,
     organism       chrom   start     end
 298  ailMel1  GL193897.1  260938  261009,
     organism       chrom   start     end
 261  ailMel1  GL193748.1  361038  361109,
    organism       chrom    start      end
 11  ailMel1  GL192354